In [1]:
import torch
import sys
import pyro

import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import pandas as pd

import numpy as np

from pooledQTL import deconvolve, beta_model, gaussian_model, io_utils, asb_data, spike_and_slab

from importlib import reload

from pathlib import Path

fdr_threshold = 0.05
device = "cuda:0" if torch.cuda.is_available() else "cpu"
use_structured_guide = True # performs much better

results_dir = Path("/gpfs/commons/home/daknowles/pooledRBPs/results/RBFOX2")
results_dir.mkdir(parents=True, exist_ok=True)

reload(spike_and_slab)

dat = pd.read_csv("../results/HNRNPA1/beta_rep0.tsv.gz", sep = "\t", index_col = False)
device = "cuda:0" if torch.cuda.is_available() else "cpu"
#device = "cpu"
#data = asb_data.ASBdata.from_pandas(dat.rename(columns = {"altCount_input" : "altCount", "totalCount_input" : "totalCount" }), device = device)

learn_likelihood_concs = True
#learn_prior_prob = True
structured_model = True

prior_prob_ase = {}
prior_prob_asb = {}

for learn_likelihood_concs in [True, False]:
    for structured_model in [True, False]:
        post_stats, post_mean = spike_and_slab.fit(
            dat, 
            learn_likelihood_concs = learn_likelihood_concs, 
            structured_model = structured_model, 
            plot = False,
            device = device
        )
        prior_prob_ase[(learn_likelihood_concs,structured_model)] = post_stats['prior_prob_ase']['mean'].cpu().item()
        prior_prob_asb[(learn_likelihood_concs,structured_model)] = post_stats['prior_prob_asb']['mean'].cpu().item()

post_stats, post_mean = spike_and_slab.fit(
    dat, 
    learn_likelihood_concs = False, 
    structured_model = True, 
    plot = True,
    device = device
)

asb_idx = post_mean['assignment_asb'].argmax()

post_mean['assignment_ase'][asb_idx]
post_mean['assignment_asb'][asb_idx]
dat.iloc[asb_idx,:]

post_stats['ase']['mean'][0,asb_idx]
post_stats['asb']['mean'][0,asb_idx]
post_stats['asb_when_ase']['mean'][0,asb_idx] # asb_when_ase

dat_sub = dat[post_mean['assignment_asb'] < 0.5]
plt.scatter(dat_sub.allelic_ratio, dat_sub.altCount_IP / dat_sub.totalCount_IP)
dat_sub = dat[post_mean['assignment_asb'] > 0.5]
plt.scatter(dat_sub.allelic_ratio, dat_sub.altCount_IP / dat_sub.totalCount_IP)


/gpfs/commons/home/daknowles/.conda/envs/asb/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/gpfs/commons/home/daknowles/pooledRBPs/src/pooledQTL/spike_and_slab.py:282: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  lambda: torch.tensor(init_dict.get(name, init), device = device),


prior_prob_ase		0.1278 6.02540
prior_prob_asb		0.001574
ase_scale		0.8193
input_count_conc		157.4
asb_scale		1.013
IP_count_conc		179.4


/gpfs/commons/home/daknowles/pooledRBPs/src/pooledQTL/spike_and_slab.py:282: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  lambda: torch.tensor(init_dict.get(name, init), device = device),


prior_prob_ase		0.1276 6.0102
prior_prob_asb		0.001829
ase_scale		0.8201
input_count_conc		157.1
asb_scale		1.037
IP_count_conc		179


/gpfs/commons/home/daknowles/pooledRBPs/src/pooledQTL/spike_and_slab.py:282: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  lambda: torch.tensor(init_dict.get(name, init), device = device),
/gpfs/commons/home/daknowles/.conda/envs/asb/lib/python3.8/site-packages/pyro/util.py:286: UserWarning: Found non-auxiliary vars in guide but not model, consider marking these infer={'is_auxiliary': True}:
{'IP_count_conc', 'input_count_conc'}
  warnings.warn(


prior_prob_ase		0.145: 6.02330
prior_prob_asb		0.001857
ase_scale		0.7887
asb_scale		1.011


/gpfs/commons/home/daknowles/pooledRBPs/src/pooledQTL/spike_and_slab.py:282: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  lambda: torch.tensor(init_dict.get(name, init), device = device),
/gpfs/commons/home/daknowles/.conda/envs/asb/lib/python3.8/site-packages/pyro/util.py:286: UserWarning: Found non-auxiliary vars in guide but not model, consider marking these infer={'is_auxiliary': True}:
{'IP_count_conc', 'input_count_conc'}
  warnings.warn(


prior_prob_ase		0.1444 6.0068
prior_prob_asb		0.002034
ase_scale		0.7887
asb_scale		1.035


NameError: name 'data' is not defined

In [3]:
prior_prob_ase

{(True, True): 0.12783393263816833,
 (True, False): 0.12758222222328186,
 (False, True): 0.14495684206485748,
 (False, False): 0.14437757432460785}

In [4]:
prior_prob_asb

{(True, True): 0.0015738141955807805,
 (True, False): 0.0018291990272700787,
 (False, True): 0.0018569828243926167,
 (False, False): 0.0020337500609457493}